In [ ]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'

max_workers = 10
max_editors = 100
email = 'd.ecer@elifesciences.org'

In [ ]:
import logging
from datetime import datetime
from functools import partial
from concurrent.futures import ThreadPoolExecutor
from typing import List

import pandas as pd
from tqdm.auto import tqdm

import data_science_pipeline.configure_warnings  # pylint: disable=unused-import
import data_science_pipeline.configure_notebook_logging  # pylint: disable=unused-import

from data_science_pipeline.utils.europepmc import (
    EuropePMCApi,
    europepmc_requests_retry_session
)
from data_science_pipeline.utils.bq import to_gbq, is_bq_not_found_exception
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

In [ ]:
LOGGER = logging.getLogger(__name__)

logging.basicConfig(level='INFO')
logging.getLogger('data_science_pipeline').setLevel(logging.INFO)

In [ ]:
editor_parsed_pubmed_links_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_links'
)

editor_parsed_pubmed_ids_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_pubmed_ids'
)

In [ ]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [ ]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [ ]:
_sql = (
    '''
    SELECT
        person_id,
        pubmed_url,
        imported_timestamp
    FROM (
        SELECT
            person_id,
            pubmed_url,
            provenance.imported_timestamp as imported_timestamp,
            ROW_NUMBER() OVER (PARTITION BY person_id ORDER BY provenance.imported_timestamp DESC) as rn
        FROM `{table}`
    ) WHERE rn = 1
    ''').format(table=editor_parsed_pubmed_ids_table_name)

try:
    existing_editor_ids_and_pubmed_url_df = read_big_query(_sql)
except Exception as e:  # pylint: disable=broad-except
    if not is_bq_not_found_exception(e):
        raise
    print('table not found: %s' % editor_parsed_pubmed_ids_table_name)
    existing_editor_ids_and_pubmed_url_df = pd.DataFrame(columns=['person_id', 'pubmed_url', 'imported_timestamp'])
print("existing_editor_ids_and_pubmed_url_df length: ", len(existing_editor_ids_and_pubmed_url_df))
existing_editor_ids_and_pubmed_url_df.head(3)

In [ ]:
existing_editor_ids_set = set(existing_editor_ids_and_pubmed_url_df['person_id'])
print("existing_editor_ids_set length :", len(existing_editor_ids_set))

In [ ]:
editor_parsed_pubmed_links_df = read_big_query(
    'SELECT * FROM `{table_name}`\nWHERE parsed_search_term IS NOT NULL'.format(
        table_name=editor_parsed_pubmed_links_table_name
    )
)
print("editor_parsed_pubmed_links_df length: ", len(editor_parsed_pubmed_links_df))
editor_parsed_pubmed_links_df.head(3)

In [ ]:
merged_editor_parsed_pubmed_links_df = editor_parsed_pubmed_links_df.merge(
    existing_editor_ids_and_pubmed_url_df,
    how='left',
    on='person_id',
    suffixes=('', '_existing')
)

print("merged_editor_parsed_pubmed_links_df length: ", len(merged_editor_parsed_pubmed_links_df))
merged_editor_parsed_pubmed_links_df.head(3)

In [ ]:
editors_with_changed_pubmed_url_df = merged_editor_parsed_pubmed_links_df[
    (merged_editor_parsed_pubmed_links_df['pubmed_url_existing'].notnull())
    &
    (
        merged_editor_parsed_pubmed_links_df['pubmed_url']
        !=
        merged_editor_parsed_pubmed_links_df['pubmed_url_existing']
    )
].drop(columns=['pubmed_url_existing', 'imported_timestamp'])

print("editors_with_changed_pubmed_url_df length: ", len(editors_with_changed_pubmed_url_df))
editors_with_changed_pubmed_url_df.head(3)

In [ ]:
editors_with_not_currently_updated_info = pd.DataFrame()
temp_df = pd.DataFrame()
for index, row in merged_editor_parsed_pubmed_links_df.iterrows():
    day_diff = abs(
        datetime.date(row['imported_timestamp'])-datetime.date(datetime.today())
    ).days
    if day_diff > 15:
        temp_df = row
    editors_with_not_currently_updated_info = editors_with_not_currently_updated_info.append(temp_df)

print("editors_with_not_currently_updated_info length: ", len(editors_with_not_currently_updated_info))
editors_with_not_currently_updated_info.head(3)

In [ ]:
new_added_editors_df = editor_parsed_pubmed_links_df[
    ~editor_parsed_pubmed_links_df['person_id'].isin(existing_editor_ids_set)
]

print("new_added_editors_df length: ", len(new_added_editors_df))
new_added_editors_df.head(3)

In [ ]:
remaining_editor_parsed_pubmed_links_df = pd.concat([
    new_added_editors_df,
    editors_with_changed_pubmed_url_df,
    editors_with_not_currently_updated_info
])

print("remaining_editor_parsed_pubmed_links_df length: ", len(remaining_editor_parsed_pubmed_links_df))
remaining_editor_parsed_pubmed_links_df.head(3)

In [ ]:
processing_editor_parsed_pubmed_links_df = remaining_editor_parsed_pubmed_links_df
if max_editors:
    processing_editor_parsed_pubmed_links_df = processing_editor_parsed_pubmed_links_df[:max_editors]
len(processing_editor_parsed_pubmed_links_df)

In [ ]:
def get_editor_pubmed_paper_ids(europepmc_api: EuropePMCApi, row) -> List[str]:
    parsed_search_term = row.parsed_search_term
    if not parsed_search_term:
        return None
    author_names = parsed_search_term.get('include', {}).get('author')
    try:
        author_names = parsed_search_term.get('include', {}).get('author')
        return europepmc_api.get_author_pmids(author_names)
    except:  # pylint: disable=bare-except
        LOGGER.error('failed to retrieve pubmed ids for author names: %s', author_names, exc_info=1)
        return None


editor_pubmed_links_result_df = processing_editor_parsed_pubmed_links_df[:max_editors].copy()

with europepmc_requests_retry_session() as session:
    europepmc_api = EuropePMCApi(
        session,
        params={'email': email}
    )
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        editor_pubmed_links_result_df['pubmed_ids'] = list(tqdm(
            executor.map(
                lambda row: get_editor_pubmed_paper_ids(europepmc_api, row),
                editor_pubmed_links_result_df.itertuples()
            ),
            total=len(editor_pubmed_links_result_df),
            leave=False
        ))

In [ ]:
non_empty_editor_pubmed_links_result_df = editor_pubmed_links_result_df[
    ~pd.isnull(editor_pubmed_links_result_df['pubmed_ids'])
].copy()
len(non_empty_editor_pubmed_links_result_df)
non_empty_editor_pubmed_links_result_df.head(3)

In [ ]:
non_empty_editor_pubmed_links_result_df['provenance'] = [{
    'source': 'europepmc',
    'imported_timestamp': datetime.utcnow().isoformat()
}] * len(non_empty_editor_pubmed_links_result_df)

In [ ]:
if len(non_empty_editor_pubmed_links_result_df) == 0:
    print('no data to upload')
else:
    print('writing to:', editor_parsed_pubmed_ids_table_name)
    to_gbq(
        non_empty_editor_pubmed_links_result_df,
        project_id=project_id,
        destination_table=editor_parsed_pubmed_ids_table_name,
        if_exists='append'
    )
    print('done')